In [25]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [26]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [27]:
!pip install -q toolz
!pip install -q wandb

In [28]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [30]:
csv = pd.read_csv("ZINC_310k.csv")

In [31]:
smiles = csv['SMILES']

In [32]:
smiles = smiles[:60000]

In [33]:
labels = torch.tensor(csv['QED'][:60000]).float()

In [34]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [35]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

In [36]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [37]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [38]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [39]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [40]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [41]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])

X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [42]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [43]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [ ]:
print("Training model...")
runner.train_gen_pred(
    X_train,
    L_train,
    X_Val,
    L_Val,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
    batch_size=conf["batch_size"],
    num_workers=conf["num_workers"],
    label_pct=0.05
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 0.000, Beta: 0.000, Loss: 110.60, KL: 373.92, MAE: 0.53291, Word Loss: 82.77, Topo Loss: 19.30, Assm Loss: 8.53, Pred Loss: 14.03, Word: 32.38, Topo: 85.16, Assm: 57.06, PNorm: 102.09, GNorm: 50.00
[Train][200] Alpha: 0.000, Beta: 0.000, Loss: 68.58, KL: 406.83, MAE: 0.26997, Word Loss: 50.04, Topo Loss: 10.68, Assm Loss: 7.86, Pred Loss: 3.98, Word: 56.29, Topo: 92.66, Assm: 60.93, PNorm: 107.18, GNorm: 42.09
[Train][300] Alpha: 0.000, Beta: 0.000, Loss: 57.56, KL: 501.76, MAE: 0.18050, Word Loss: 41.02, Topo Loss: 9.15, Assm Loss: 7.40, Pred Loss: 1.89, Word: 64.55, Topo: 93.61, Assm: 63.43, PNorm: 110.76, GNorm: 42.52
[Train][400] Alpha: 0.000, Beta: 0.000, Loss: 50.76, KL: 604.05, MAE: 0.17801, Word Loss: 35.98, Topo Loss: 7.92, Assm Loss: 6.85, Pred Loss: 1.93, Word: 68.50, Topo: 94.40, Assm: 65.57, PNorm: 114.45, GNorm: 50.00
learning rate: 0.000729
[Train][500] Alpha: 0.000, Beta: 0.010, Loss: 47.54, KL: 366.21, MAE: 0.1

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)